In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
data = pd.read_csv('delivery_orders_march.csv', header=0)

In [3]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


# Time

-change time format to UTC

-Singapore's time zone is UTC+8

In [4]:
data['pick_date'] = data['pick'].apply(lambda x: (datetime.utcfromtimestamp(x) + timedelta(hours=8)))
data['pick_date'] = data['pick_date'].apply(lambda x: x.date())
data['first'] = data['1st_deliver_attempt'].apply(lambda x: (datetime.utcfromtimestamp(x)+ timedelta(hours=8)).date())
data['second'] = data['2nd_deliver_attempt'].apply(lambda x: (pd.Timestamp(x, unit='s')+ timedelta(hours=8)).date())

In [5]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,pick_date,first,second
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-02,2020-03-05,NaT
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-04,2020-03-06,2020-03-10
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-04,2020-03-06,NaT
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-05,2020-03-07,NaT
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-04,2020-03-06,NaT


# Calculate divery time for first and second attempts

In [6]:
data['first_days'] = data['first'] - data['pick_date']
data['second_days'] = data['second'] - data['first']

In [7]:
holidays = ['2020-03-01',
            '2020-03-08', 
            '2020-03-15', 
            '2020-03-22', 
            '2020-03-25', 
            '2020-03-29', 
            '2020-03-30', 
            '2020-03-31', 
            '2020-04-05']
h_days = [datetime.strptime(i, '%Y-%m-%d').date() for i in holidays]

In [8]:
data['h_first'] = 0
data['h_second'] = 0
for i in h_days:
    data.loc[(data['pick_date'] <= i) & (i <= data['first']),'h_first'] += 1
    data.loc[(data['first']  <= i) & (i <= data['second']),'h_second'] += 1

In [9]:
data['h_first'] = data['h_first'].apply(lambda x: timedelta(days=x))
data['h_second'] = data['h_second'].apply(lambda x: timedelta(days=x))

In [10]:
data['final_first'] = data['first_days'] - data['h_first']
data['final_second'] = data['second_days'] - data['h_second']

In [11]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,pick_date,first,second,first_days,second_days,h_first,h_second,final_first,final_second
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-02,2020-03-05,NaT,3 days,NaT,0 days,0 days,3 days,NaT
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-04,2020-03-06,2020-03-10,2 days,4 days,0 days,1 days,2 days,3 days
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-04,2020-03-06,NaT,2 days,NaT,0 days,0 days,2 days,NaT
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-05,2020-03-07,NaT,2 days,NaT,0 days,0 days,2 days,NaT
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-04,2020-03-06,NaT,2 days,NaT,0 days,0 days,2 days,NaT


# Calculate standard delivery time

In [12]:
data['b_addr'] = data['buyeraddress'].apply(lambda x: x.split(' ')[-1])
data['s_addr'] = data['selleraddress'].apply(lambda x: x.split(' ')[-1])

In [13]:
data.loc[(data['b_addr'].str.contains('manila|Manila')), 'b_city'] = "Metro Manila"
data.loc[(data['b_addr'].str.contains('luzon|Luzon')), 'b_city'] = "Luzon"
data.loc[(data['b_addr'].str.contains('visayas|Visayas')), 'b_city'] = "Visayas"
data.loc[(data['b_addr'].str.contains('mindanao|Mindanao')), 'b_city'] = "Mindanao"

In [14]:
data.loc[(data['s_addr'].str.contains('manila|Manila')), 's_city'] = "Metro Manila"
data.loc[(data['s_addr'].str.contains('luzon|Luzon')), 's_city'] = "Luzon"
data.loc[(data['s_addr'].str.contains('visayas|Visayas')), 's_city'] = "Visayas"
data.loc[(data['s_addr'].str.contains('mindanao|Mindanao')), 's_city'] = "Mindanao"

In [15]:
data['sla'] = timedelta(days=7)
data.loc[(data['b_city']=='Metro Manila') & (data['s_city']=='Metro Manila'), 'sla'] = timedelta(days=3)
data.loc[(data['b_city']=='Luzon') & (data['s_city']=='Luzon'), 'sla'] = timedelta(days=5)
data.loc[(data['b_city']=='Metro Manila') & (data['s_city']=='Luzon'), 'sla'] = timedelta(days=5)
data.loc[(data['b_city']=='Luzon') & (data['s_city']=='Metro Manila'), 'sla'] = timedelta(days=5)

In [16]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,pick_date,first,second,first_days,second_days,h_first,h_second,final_first,final_second,b_addr,s_addr,b_city,s_city,sla
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-02,2020-03-05,NaT,3 days,NaT,0 days,0 days,3 days,NaT,Manila,Manila,Metro Manila,Metro Manila,3 days
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-04,2020-03-06,2020-03-10,2 days,4 days,0 days,1 days,2 days,3 days,Manila,Manila,Metro Manila,Metro Manila,3 days
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-04,2020-03-06,NaT,2 days,NaT,0 days,0 days,2 days,NaT,manila,Manila,Metro Manila,Metro Manila,3 days
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-05,2020-03-07,NaT,2 days,NaT,0 days,0 days,2 days,NaT,Manila,Manila,Metro Manila,Metro Manila,3 days
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-04,2020-03-06,NaT,2 days,NaT,0 days,0 days,2 days,NaT,luzon,Manila,Luzon,Metro Manila,5 days


In [17]:
df_final = data[['orderid', 'final_first', 'final_second', 'sla']]

In [18]:
df_final.head()

,orderid,final_first,final_second,sla
0,2215676524,3 days,NaT,3 days
1,2219624609,2 days,3 days,3 days
2,2220979489,2 days,NaT,3 days
3,2221066352,2 days,NaT,3 days
4,2222478803,2 days,NaT,5 days


In [19]:
df_final['is_late'] = 0
df_final.loc[df_final['final_first']>df_final['sla'], 'is_late'] = 1
df_final.loc[df_final['final_second']>timedelta(days=3), 'is_late'] = 1

/Users/ameliatsai/Desktop/Data Science/Kaggle/Competition/kaggleenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/ameliatsai/Desktop/Data Science/Kaggle/Competition/kaggleenv/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [20]:
df_final['is_late'].sum()/len(df_final)

0.2400336490767755

In [21]:
df_final[df_final.is_late==1]

,orderid,final_first,final_second,sla,is_late
7,2224695304,7 days,NaT,3 days,1
19,2227790841,7 days,7 days,7 days,1
21,2227816770,4 days,1 days,3 days,1
22,2227817800,8 days,NaT,7 days,1
24,2227831996,4 days,3 days,3 days,1
...,...,...,...,...,...
3175495,31140751864621,5 days,NaT,3 days,1
3175526,31151634009261,4 days,NaT,3 days,1
3175569,31188333685284,4 days,NaT,3 days,1
3175668,31244254829792,4 days,NaT,3 days,1


In [22]:
result = df_final[['orderid', 'is_late']]

In [23]:
result.to_csv('submit.csv', index=False)